In [1]:
import pandas as pd
import numpy as np

In [2]:
#### Load in the dataset

micro_growing = pd.read_csv("/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/data/ct_sensory_micro.csv", sep=",", header=0)

micro_growing.head()

,Eartag_Number,Flavour,Juicy,Tenderness,Tenderness2,OffFlavour,Acceptability,ProductCodeID,ProductCode,Steak_Number,...,ct_rumen,fat_kg,muscle_kg,bone_kg,gigot,EMA,INF,rumen,ct_date_diff,sensory_date_diff
0,372042060107286,5.4,6.2,7.6,2.0,1.0,NaN,9690,897,H,...,5.839,1.0,6.0,2.5,3.0,10.5,1.5,6.0,4.0,76.041667
1,372042060107353,5.4,5.8,6.2,1.6,1.0,NaN,9560,644,H,...,5.623,2.0,6.5,2.5,3.5,12.0,2.5,5.5,2.0,51.041667
2,372042060107364,5.6,5.6,6.6,2.0,1.0,NaN,9831,892,H,...,4.081,1.0,6.5,2.5,3.5,10.0,1.0,4.0,2.0,103.041667
3,372042060107391,5.0,5.4,5.6,1.4,1.0,NaN,9684,538,H,...,4.109,1.0,6.0,2.5,3.0,12.0,1.5,4.0,2.0,72.041667
4,372042060107410,6.8,6.2,5.8,1.4,1.0,NaN,9545,892,H,...,4.891,1.0,6.0,2.5,3.0,10.0,1.0,5.0,2.0,50.041667


In [3]:
## Need to find which columns are microbial genera
for i, col in enumerate(micro_growing.columns):
    print(f"{i}: {col}")

0: Eartag_Number
1: Flavour
2: Juicy
3: Tenderness
4: Tenderness2
5: OffFlavour
6: Acceptability
7: ProductCodeID
8: ProductCode
9: Steak_Number
10: Comments
11: OffFlavourComment
12: sensory_date
13: VID
14: X0319.6G20
15: Abditibacterium
16: Absconditabacteriales_.SR1.
17: Acetanaerobacterium
18: Acetitomaculum
19: Acetobacteroides
20: Acinetobacter
21: Actinobacillus
22: Actinomyces
23: Agathobacter
24: Agromyces
25: Akkermansia
26: Aliterella
27: Alkalibacter
28: Allisonella
29: Alloprevotella
30: Allorhizobium.Neorhizobium.Pararhizobium.Rhizobium
31: Alysiella
32: Amnipila
33: Anaerobiospirillum
34: Anaerofustis
35: Anaeroplasma
36: Anaerostipes
37: Anaerovibrio
38: Anaerovorax
39: Asteroleplasma
40: Atopobium
41: Bacillus
42: Bacteroidales_BS11_gut_group
43: Bacteroidales_RF16_group
44: Bacteroidales_UCG.001
45: Bacteroides
46: Bacteroidetes_BD2.2
47: Barnesiella
48: Bdellovibrio
49: Bergeyella
50: Bibersteinia
51: Blastocatellaceae
52: Blautia
53: Bosea
54: Bradymonadales
55: Br

In [7]:
#### Filtering the dataset #####

# Columns wherein microbial genera are
genus_cols = micro_growing.columns[14:323]
abund = micro_growing[genus_cols].copy()

# Pull in ANI_ID from column 324
abund["ANI_ID"] = micro_growing.iloc[:, 324].values

# ------------------------------------------------------------------
# 3.  PREVALENCE FILTER  (>10 % non-zero)
# ------------------------------------------------------------------
prev = (abund[genus_cols] > 0).mean()
keep = prev[prev >= 0.10].index.tolist()

print(f"Kept {len(keep)} genera out of {len(genus_cols)} (≥10 % prevalence)")


Kept 160 genera out of 309 (≥10 % prevalence)


In [10]:
# ------------------------------------------------------------------
# 4. LOG TRANSFORM with small pseudocount
# ------------------------------------------------------------------
abund_log = abund.copy()
abund_log[keep] = np.log(abund_log[keep] + 1e-6)

# Keep only the transformed genera + ANI_ID
abund_log = abund_log[keep + ["ANI_ID"]]

abund_log.head()

,X0319.6G20,Absconditabacteriales_.SR1.,Acetitomaculum,Agromyces,Alloprevotella,Allorhizobium.Neorhizobium.Pararhizobium.Rhizobium,Alysiella,Anaerobiospirillum,Anaeroplasma,Anaerostipes,...,X.Eubacterium._ventriosum_group,X.Eubacterium._xylanophilum_group,X.Ruminococcus._gauvreauii_group,X.Ruminococcus._gnavus_group,hoa5.07d05_gut_group,p.251.o5,probable_genus_10,uncultured,vadinBE97,ANI_ID
0,-2.995712,-1.427112,-13.815511,-2.813394,-13.815511,-13.815511,-13.815511,-13.815511,-2.525716,-13.815511,...,-13.815511,-3.218851,-3.911973,-13.815511,-13.815511,-13.815511,-13.815511,0.854416,-3.911973,1087987
1,-2.040213,-0.693145,-13.815511,-13.815511,-4.605070,-3.911973,-13.815511,-13.815511,-2.120255,-13.815511,...,-2.813394,-3.218851,-3.911973,-13.815511,-13.815511,-13.815511,-2.207266,0.570980,-2.995712,1088072
2,-2.040213,-2.525716,-13.815511,-13.815511,-13.815511,-13.815511,-13.815511,-13.815511,-4.605070,-13.815511,...,-4.605070,-2.813394,-13.815511,-13.815511,-13.815511,-13.815511,-13.815511,1.226713,-4.605070,1088056
3,-4.605070,-1.771951,-13.815511,-4.605070,-4.605070,-13.815511,-13.815511,-13.815511,-3.218851,-13.815511,...,-2.659246,-3.911973,-13.815511,-13.815511,-2.995712,-13.815511,-13.815511,0.810931,-3.218851,1088090
4,-4.605070,-2.302575,-3.506525,-3.506525,-3.911973,-3.911973,-13.815511,-13.815511,-1.771951,-13.815511,...,-2.813394,-4.605070,-13.815511,-13.815511,-13.815511,-13.815511,-13.815511,1.226713,-13.815511,1088109


In [14]:
# Now we can drop raw genera abundances and merge back in these new filtered CLR

# Drop raw values
micro_clean = micro_growing.drop(columns=micro_growing.columns[14:323])

# Merge on ANI_ID
micro_final = micro_clean.merge(abund_log, on="ANI_ID")

micro_final.head()

,Eartag_Number,Flavour,Juicy,Tenderness,Tenderness2,OffFlavour,Acceptability,ProductCodeID,ProductCode,Steak_Number,...,X.Eubacterium._ruminantium_group,X.Eubacterium._ventriosum_group,X.Eubacterium._xylanophilum_group,X.Ruminococcus._gauvreauii_group,X.Ruminococcus._gnavus_group,hoa5.07d05_gut_group,p.251.o5,probable_genus_10,uncultured,vadinBE97
0,372042060107286,5.4,6.2,7.6,2.0,1.0,NaN,9690,897,H,...,-2.525716,-13.815511,-3.218851,-3.911973,-13.815511,-13.815511,-13.815511,-13.815511,0.854416,-3.911973
1,372042060107353,5.4,5.8,6.2,1.6,1.0,NaN,9560,644,H,...,-13.815511,-2.813394,-3.218851,-3.911973,-13.815511,-13.815511,-13.815511,-2.207266,0.570980,-2.995712
2,372042060107364,5.6,5.6,6.6,2.0,1.0,NaN,9831,892,H,...,-2.302575,-4.605070,-2.813394,-13.815511,-13.815511,-13.815511,-13.815511,-13.815511,1.226713,-4.605070
3,372042060107391,5.0,5.4,5.6,1.4,1.0,NaN,9684,538,H,...,-2.407934,-2.659246,-3.911973,-13.815511,-13.815511,-2.995712,-13.815511,-13.815511,0.810931,-3.218851
4,372042060107410,6.8,6.2,5.8,1.4,1.0,NaN,9545,892,H,...,-1.469672,-2.813394,-4.605070,-13.815511,-13.815511,-13.815511,-13.815511,-13.815511,1.226713,-13.815511


In [18]:
print(micro_final.shape)

(257, 513)


In [19]:
# save as csv
# Define output path
output_path = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/data/CLR_micro.csv"

# Save without the index column
micro_final.to_csv(output_path, index=False)

print(f"Saved to: {output_path}")


Saved to: /home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/data/CLR_micro.csv


In [20]:
# Also save abund_log as will be useful for matrix for LMM

output_path = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/data/CLR_counts_only.csv"

# Save without the index column
abund_log.to_csv(output_path, index=False)

print(f"Saved to: {output_path}")

Saved to: /home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/data/CLR_counts_only.csv
